In [32]:
import gym
import numpy as np

In [33]:
def choose_action(state, epsilon):
    if np.random.uniform(0, 1) < epsilon:  # np.random.uniform(0, 1) include 0 but not include 1
        action = env.action_space.sample()
    else:
        action = np.argmax(Q[state, :])
    return action


def update(Q, state, action, reward, state2, action2, alpha, gamma):
    increment = reward + gamma * Q[state2, action2] - Q[state, action]
    Q[state, action] = Q[state, action] + alpha * increment

In [34]:
def sarsa(Q, epsilon, alpha, gamma, total_episodes, max_steps):
    for episode in range(total_episodes):
        t = 0
        state1 = env.reset()[0]
        action1 = choose_action(state1, epsilon)
        while t < max_steps:
            # Getting the next state
            state2, reward, is_truncated, is_finished, prob = env.step(action1)
            # If there is a hole, the transition probability of walking into a hole = 0
            # keep the state unchanged
            if is_truncated and reward == 0:
                env.P[state1][action1][0] = (1.0, state1, 0.0, False)
                break
            # Choosing the next action
            action2 = choose_action(state2, epsilon)
            update(Q, state1, action1, reward, state2, action2, alpha, gamma)
            state1 = state2
            action1 = action2
            t += 1
            # (for test)If reaching the terminal state
            # if reward == 1 and is_truncated:
            #     print(episode)
            #     print(Q)
    return Q

In [35]:
def get_optimal_policy(Q):
    env_new = gym.make("FrozenLake-v1", render_mode = "human", is_slippery=False)
    state = env_new.reset()[0]
    optimal_policy = []
    while True:
        optimal_action = np.argmax(Q[state, :])
        optimal_policy.append(optimal_action)
        state, reward, is_truncated, is_finished, prob = env_new.step(optimal_action)
        if reward == 1 and is_truncated:
            return optimal_policy

In [36]:
env = gym.make("FrozenLake-v1", is_slippery=False)
epsilon = 0.9
alpha = 0.2  # control the increment
gamma = 0.95  # the decay of reward
Q = np.zeros([env.observation_space.n, env.action_space.n])
Q_value = sarsa(Q, epsilon, alpha, gamma, total_episodes=500, max_steps=50)
print("The Q value after Sarsa method")
print(Q_value)
optimal_policy = get_optimal_policy(Q_value)
print("The optimal policy is: ")
print(optimal_policy)

The Q value after Sarsa method
[[0.23292494 0.272599   0.22702243 0.22947438]
 [0.22460372 0.23273751 0.26209873 0.22384481]
 [0.22398541 0.31803239 0.22934295 0.28237667]
 [0.24235771 0.23764444 0.22635399 0.23752137]
 [0.29574451 0.35925586 0.29345126 0.22505131]
 [0.         0.         0.         0.        ]
 [0.33803015 0.41221205 0.36992291 0.27916388]
 [0.         0.         0.         0.        ]
 [0.36140443 0.32523305 0.42378855 0.28213433]
 [0.34447432 0.57960295 0.45365781 0.41773757]
 [0.45126679 0.54470332 0.42467018 0.35525506]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.53207863 0.494543   0.71014097 0.42031912]
 [0.48252592 0.56787356 1.         0.46858195]
 [0.         0.         0.         0.        ]]
The optimal policy is: 
[1, 1, 2, 1, 2, 2]
